In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
df = pd.read_csv('./data/training_dataset.csv')

In [3]:
df = df.drop_duplicates().dropna()
df['year'] = df['year'].astype(float)
x_vars = ['currentRatio', 'grossProfitMargin', 'debtEquityRatio', 'assetTurnover', 'operatingCashFlowPerShare','year','sp500','n_sector']
X = df[x_vars]
y = df[['n_rating']]


val_idxs = np.random.randint(low=1,high=X.shape[0],size=int(X.shape[0]*0.20))

X.drop(val_idxs, inplace=True)
y.drop(val_idxs, inplace=True)
y = y.values.T[0]

C:\Users\Lucas\miniconda3\envs\polienv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [4]:
knnModel = KNeighborsClassifier(n_neighbors=20)
knnModel.fit(X,y.T)

KNeighborsClassifier(n_neighbors=20)

In [5]:
df.loc[val_idxs[0], x_vars].values

array([1.22185129, 1.0, 3.18953043, 0.445110389, 13.22604423, 2016.0, 0.0,
       5], dtype=object)

In [6]:
yPred = []
yTrue = []
for idx in val_idxs:
    val_x = [float(x) for x in df.loc[idx, x_vars].values]
    yPred.append(knnModel.predict([val_x])[0])
    yTrue.append(df.loc[idx, 'n_rating'])

In [7]:
mean_absolute_error(yTrue,yPred)

0.9175257731958762

In [8]:
yPredSparse = np.zeros((len(yPred),7))
for i, c in enumerate(list(yPred)):
    yPredSparse[i][c] = 1.0

In [9]:
roc_auc_score(yTrue,np.array(yPredSparse),multi_class='ovr',average='weighted')

0.560323099163443